In [90]:
#importação de bibliotecas
import pandas as pd
from pandas_datareader import wb
import numpy as np
import math

In [2]:
# Definição dos dados (BancoMundial)
codigos = ["NE.EXP.GNFS.CD", "NE.IMP.GNFS.CD", "NV.AGR.TOTL.CD", "NY.GDP.MKTP.CD", "NE.RSB.GNFS.CD"]

Informação sobre cada dado:


> NE.EXP.GNFS.CD: Exportação de bens e serviços (em dólar).

> NE.IMP.GNFS.CD: Importação de bens e serviços (em dólar).

> NV.AGR.TOTL.CD: Valor agregado de agricultura, silvicultura e pesca.

> NY.GDP.MKTP.CD: PIB.

> NE.RSB.GNFS.CD: Resultado líquido dos serviços comerciais.











In [46]:
# Download dos dados e criação de um dataframe
data = wb.download(country = "all", indicator = codigos, start = 2019, end = 2019).reset_index()

In [47]:
data.head()

,country,year,NE.EXP.GNFS.CD,NE.IMP.GNFS.CD,NV.AGR.TOTL.CD,NY.GDP.MKTP.CD,NE.RSB.GNFS.CD
0,Africa Eastern and Southern,2019,2.391056e+11,2.702334e+11,1.269453e+11,1.000834e+12,NaN
1,Africa Western and Central,2019,1.762708e+11,2.119928e+11,1.711539e+11,8.225384e+11,NaN
2,Arab World,2019,1.248196e+12,1.093191e+12,1.399285e+11,2.868891e+12,NaN
3,Caribbean small states,2019,NaN,NaN,3.129407e+09,7.728280e+10,NaN
4,Central Europe and the Baltics,2019,1.041857e+12,1.000260e+12,4.609954e+10,1.674114e+12,NaN


In [48]:
# Obtém a lista de paises presentes
paises = wb.get_countries()

In [49]:
#retirando agregações e selecionando apenas os nomes:
paises = paises[paises["region"] != "Aggregates"]
paises_nomes= paises['name']

In [50]:
paises_nomes.head()

0          Aruba
2    Afghanistan
5         Angola
6        Albania
7        Andorra
Name: name, dtype: object

In [51]:
#deixando apenas informações nos dados onde há o nome de um país:
df = data[data["country"].isin(paises_nomes)].dropna()

In [52]:
df.head()

,country,year,NE.EXP.GNFS.CD,NE.IMP.GNFS.CD,NV.AGR.TOTL.CD,NY.GDP.MKTP.CD,NE.RSB.GNFS.CD
50,Albania,2019,4.821429e+09,6.926960e+09,2.832293e+09,1.540183e+10,-2.105530e+09
51,Algeria,2019,3.901433e+10,4.997396e+10,2.118960e+10,1.717603e+11,-1.095964e+10
54,Angola,2019,2.827171e+10,1.180943e+10,5.463377e+09,6.930911e+10,1.646228e+10
55,Antigua and Barbuda,2019,1.196203e+09,1.156390e+09,2.925185e+07,1.675404e+09,3.981337e+07
56,Argentina,2019,8.025948e+10,6.584563e+10,2.381408e+10,4.477547e+11,1.441386e+10


In [54]:
#separando os nomes dos países e seus indicadores
x = df.drop(['year','country'],axis=1)
y = df['country']

In [67]:
#normalização dos dados, separando todos os indicadores em um array, e referenciando o mesmo por uma chave
#chave essa sendo o nome do país!

dict_countries = {}
for i in range(len(df)):
  dict_countries[y.iloc[i]] = np.array(x.iloc[i])

In [71]:
dict_countries['Brazil']

array([ 2.64562973e+11,  2.76634807e+11,  7.87720311e+10,  1.87328816e+12,
       -1.20718340e+10])

In [91]:
#no caso queremos analisar o desempenho do Brasil em relação aos outros países, para isso iremos
#inicialmente utilizar o método da distância euclideana!

new_dict = {}
for country in dict_countries:
    if country == 'Brazil':
      pass
    else:
      euclidean_distance = math.dist(dict_countries['Brazil'],dict_countries[country])
      cosine_similarity = np.dot(dict_countries['Brazil'],dict_countries[country])/(np.linalg.norm(dict_countries['Brazil'])*np.linalg.norm(dict_countries[country]))
      new_dict[country] = [euclidean_distance,cosine_similarity]

In [92]:
#Transformando o dicionário resultante em um Dataframe (para facilitar visiualização)

df_new = pd.DataFrame(new_dict)
df_new = df_new.T
df_new.set_axis(['euclidean_distance_BR', 'cosine_similarity_BR'], axis='columns', inplace=True)
df_new.head()

<ipython-input-92-65d7e30ed0a4>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df_new.set_axis(['euclidean_distance_BR', 'cosine_similarity_BR'], axis='columns', inplace=True)


,euclidean_distance_BR,cosine_similarity_BR
Albania,1.896791e+12,0.942001
Algeria,1.732271e+12,0.984082
Angola,1.840243e+12,0.946549
Antigua and Barbuda,1.911683e+12,0.835655
Argentina,1.454052e+12,0.998565


In [93]:
# Ordenar o DataFrame por idade em ordem decrescente (um leva em conta a distância euclideana o outro a similaridade de cossenos)
df_ordenado_ED = df_new.sort_values(by='euclidean_distance_BR', ascending=False)
df_ordenado_CS = df_new.sort_values(by='cosine_similarity_BR', ascending=False)

#Visualização Final: TOP 5 países mais similares a economia brasileira em 2019!

In [94]:
df_ordenado_ED.head()

,euclidean_distance_BR,cosine_similarity_BR
United States,1.985247e+13,0.999012
China,1.285882e+13,0.998332
Japan,3.365125e+12,0.998604
Germany,2.873396e+12,0.935519
Kiribati,1.913451e+12,0.688861


In [95]:
df_ordenado_CS.head()

,euclidean_distance_BR,cosine_similarity_BR
United States,1.985247e+13,0.999012
Cuba,1.808944e+12,0.998754
Japan,3.365125e+12,0.998604
Argentina,1.454052e+12,0.998565
China,1.285882e+13,0.998332


#Visualização final: TOP 5 países menos similares a economia brasileira em 2019!

In [96]:
df_ordenado_ED.tail()

,euclidean_distance_BR,cosine_similarity_BR
"Korea, Rep.",5.557814e+11,0.957158
Italy,4.999177e+11,0.978997
Australia,4.917272e+11,0.992870
Canada,4.551362e+11,0.971913
Russian Federation,3.248566e+11,0.986538


In [97]:
df_ordenado_CS.tail()

,euclidean_distance_BR,cosine_similarity_BR
Malta,1.891305e+12,0.579599
Djibouti,1.909243e+12,0.576395
Singapore,1.579082e+12,0.569907
"Hong Kong SAR, China",1.601100e+12,0.549017
Luxembourg,1.816264e+12,0.524871
